<a href="https://colab.research.google.com/github/nicolasJimenez11/paradigmas-/blob/main/PERROSYGATOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import zipfile, os

# Rutas ZIP y extract
train_zip_path = '/content/dataset/train.zip'
train_extract_path = '/content/dataset/train_extracted'

os.makedirs(train_extract_path, exist_ok=True)
with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
    zip_ref.extractall(train_extract_path)

# Detectar subcarpeta (debería ser ['train'])
subdirs = [d for d in os.listdir(train_extract_path)
           if os.path.isdir(os.path.join(train_extract_path, d))]
if len(subdirs) == 1:
    raw_train_dir = os.path.join(train_extract_path, subdirs[0])
else:
    raw_train_dir = train_extract_path

print("Imágenes en:", raw_train_dir)
print("Primeros archivos:", os.listdir(raw_train_dir)[:10])


Imágenes en: /content/dataset/train_extracted/train
Primeros archivos: ['dog.11258.jpg', 'dog.6608.jpg', 'dog.8094.jpg', 'cat.11065.jpg', 'dog.11225.jpg', 'dog.8114.jpg', 'cat.2826.jpg', 'dog.9025.jpg', 'dog.1616.jpg', 'cat.3866.jpg']


In [63]:
import os

base_split = '/content/dataset_splitted'
train_dir = os.path.join(base_split, 'train')
val_dir   = os.path.join(base_split, 'validation')

for folder in [train_dir, val_dir]:
    for cls in ['cats', 'dogs']:
        os.makedirs(os.path.join(folder, cls), exist_ok=True)

print("Estructura de carpetas creada en", base_split)


Estructura de carpetas creada en /content/dataset_splitted


In [70]:
import random, shutil

# Listar solo archivos de imagen
all_images = [f for f in os.listdir(raw_train_dir)
              if f.lower().endswith(('.jpg','.jpeg','.png'))]
random.shuffle(all_images)

# División 80/20
split_idx    = int(0.8 * len(all_images))
train_imgs   = all_images[:split_idx]
val_imgs     = all_images[split_idx:]

def mover(files, src, dst):
    for fname in files:
        low = fname.lower()
        if low.startswith('cat'):
            cls = 'cats'
        elif low.startswith('dog'):
            cls = 'dogs'
        else:
            continue
        src_path = os.path.join(src, fname)
        dst_path = os.path.join(dst, cls, fname)
        shutil.copy2(src_path, dst_path)
        print(f"{fname} → {dst_path}")

print("Moviendo entrenamiento...")
mover(train_imgs, raw_train_dir, train_dir)
print("Moviendo validación...")
mover(val_imgs, raw_train_dir, val_dir)


Se han truncado las últimas 5000 líneas del flujo de salida.
cat.8833.jpg → /content/dataset_splitted/validation/cats/cat.8833.jpg
cat.5923.jpg → /content/dataset_splitted/validation/cats/cat.5923.jpg
cat.6462.jpg → /content/dataset_splitted/validation/cats/cat.6462.jpg
dog.4796.jpg → /content/dataset_splitted/validation/dogs/dog.4796.jpg
dog.11558.jpg → /content/dataset_splitted/validation/dogs/dog.11558.jpg
dog.110.jpg → /content/dataset_splitted/validation/dogs/dog.110.jpg
dog.6447.jpg → /content/dataset_splitted/validation/dogs/dog.6447.jpg
cat.1787.jpg → /content/dataset_splitted/validation/cats/cat.1787.jpg
dog.10763.jpg → /content/dataset_splitted/validation/dogs/dog.10763.jpg
cat.11929.jpg → /content/dataset_splitted/validation/cats/cat.11929.jpg
dog.2767.jpg → /content/dataset_splitted/validation/dogs/dog.2767.jpg
cat.6889.jpg → /content/dataset_splitted/validation/cats/cat.6889.jpg
dog.9035.jpg → /content/dataset_splitted/validation/dogs/dog.9035.jpg
dog.10334.jpg → /content/

In [65]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [66]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │    18,940,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,034,177 (72.61 MB)

 Trainable params: 19,034,177 (72.61 MB)

 Non-trainable params: 0 (0.00 B)

In [67]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1335s 2s/step - accuracy: 0.5801 - loss: 0.7797 - val_accuracy: 0.6470 - val_loss: 0.6224
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1341s 2s/step - accuracy: 0.6910 - loss: 0.5897 - val_accuracy: 0.7474 - val_loss: 0.5086
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1344s 2s/step - accuracy: 0.7487 - loss: 0.5156 - val_accuracy: 0.7672 - val_loss: 0.4877
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1302s 2s/step - accuracy: 0.7772 - loss: 0.4646 - val_accuracy: 0.8047 - val_loss: 0.4198
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1382s 2s/step - accuracy: 0.8106 - loss: 0.4196 - val_accuracy: 0.8199 - val_loss: 0.3911
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1342s 2s/step - accuracy: 0.8290 - loss: 0.3867 - val_accuracy: 0.8355 - val_loss: 0.3704
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1291s 2s/step - accuracy: 0.8405 - loss: 0.3639 - val_accuracy: 0.8249 - val_loss: 0.3833
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1349s 2s/step - accuracy: 0.8514 - loss: 0.3440 - 

In [69]:
loss, acc = model.evaluate(
    validation_generator,
    steps=validation_generator.samples // validation_generator.batch_size
)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {acc:.4f}")


156/156 ━━━━━━━━━━━━━━━━━━━━ 77s 494ms/step - accuracy: 0.8697 - loss: 0.3124
Validation Loss: 0.3136
Validation Accuracy: 0.8694
